In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from sklearn.metrics import accuracy_score, f1_score
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Reshape
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
def loadData(file):
    with open(file, 'r', encoding="utf8") as f:
        data = f.readlines()
    result = []
    for d in data:
        d = d.strip()
        if (len(d) > 0):
            result.append(d)
    return result
def print_result(y_pred, y_test, clf_name):
    ACC = accuracy_score(y_pred, y_test)
    F1 = f1_score(y_pred, y_test, average='macro')
    print("%s\t(accuracy, f1) = (%.5f, %.5f)"%(clf_name, ACC, F1))
def process_raw_data(normal_data, anomalous_data):
    # create dict
    char_dict = {}
    char_smpl = ' '.join(anomalous_data)
    char_smpl = sorted(list(set(char_smpl)))
    for idx, ch in enumerate(char_smpl):
        char_dict[ch] = idx
    # convert
    normal_data = [[char_dict[el] for el in line] for line in normal_data]
    anomalous_data = [[char_dict[el] for el in line] for line in anomalous_data]
    # merge data and create target data
    data = normal_data + anomalous_data
    # train_target = np.ones(len(normal_data)).tolist() + np.zeros(len(anomalous_data)).tolist()
    target = [1]*len(normal_data) + [0]*len(anomalous_data)
    print('Good requests:', len(normal_data))
    print('Bad requests:', len(anomalous_data))
    print('Total requests:', len(target))
    # set max len element of data
    for i in range(len(data)):
        if (len(data[i]) < 300):
            data[i] = data[i] + [0]*(300 - len(data[i]))
        else:
            data[i] = data[i][:300]
    # split
    train_data, test_data, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 22)
    train_data, val_data, y_train, y_val = train_test_split(train_data, y_train, test_size=0.25, random_state = 11)
    # one-hot vector
    X_train = np.asarray([to_categorical(i, num_classes=63) for i in train_data])
    X_val = np.asarray([to_categorical(i, num_classes=63) for i in val_data])
    X_test = np.asarray([to_categorical(i, num_classes=63) for i in test_data])
    # print
    print("Requests for Train: ", len(y_train))
    print("Requests for Validation: ", len(y_val))
    print("Requests for Test: ", len(y_test))
    print("Split Train:Validation:Test = 6:2:2")
    return X_train, X_val, X_test, y_train, y_val, y_test

In [0]:
normal_data = loadData('normalRequest.txt')
anomalous_data = loadData('anomalousRequest.txt')

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test = process_raw_data(normal_data, anomalous_data)

Good requests: 36000
Bad requests: 25065
Total requests: 61065
Requests for Train:  36639
Requests for Validation:  12213
Requests for Test:  12213
Split Train:Validation:Test = 6:2:2


In [5]:
shape = X_train.shape
print(shape)

(36639, 300, 63)


## Mạng CNN cơ bản với 3 tầng Conv2D và độ rộng là 32

In [6]:
model = Sequential()
model.add(Reshape((shape[1], shape[2], 1), input_shape=(shape[1], shape[2])))

model.add(Conv2D(32, (3, 63), activation='relu'))
model.add(Conv2D(32, (3, 1), activation='relu'))
model.add(Conv2D(32, (3, 1), activation='relu'))

model.add(Dropout(0.25))
model.add(GlobalMaxPooling2D())

model.add(Dense(64))
model.add(Dense(1, activation='sigmoid'))

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

y_pred = model.predict_classes(X_test)
print_result(y_pred, y_test, 'CNN Conv2d: ')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 300, 63, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 298, 1, 32)        6080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 296, 1, 32)        3104      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 294, 1, 32)        3104      
_________________________________________________________________
dropout_1 (Dropout)          (None, 294, 1, 32)        0         
_________________________________________________________________
global_max_pooling2d_

## Model 2 so với model 1 có 5 lớp Conv2D
Nhưng kết quả đạt được không hiệu quả hơn so với model 1
### Kết luận: Số tầng Conv2D đến một mức nào đó đã là đạt hiệu quả cao nhất, tăng thêm cũng không đạt được hiệu quả nữa

In [7]:
model2 = Sequential()
model2.add(Reshape((shape[1], shape[2], 1), input_shape=(shape[1], shape[2])))

model2.add(Conv2D(32, (3, 63), activation='relu'))
model2.add(Conv2D(32, (3, 1), activation='relu'))
model2.add(Conv2D(32, (3, 1), activation='relu'))
model2.add(Conv2D(32, (3, 1), activation='relu'))
model2.add(Conv2D(32, (3, 1), activation='relu'))

model2.add(Dropout(0.25))
model2.add(GlobalMaxPooling2D())

model2.add(Dense(64))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()
model2.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['acc'])

model2.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

y_pred2 = model2.predict_classes(X_test)
print_result(y_pred2, y_test, 'CNN Conv2d: ')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 300, 63, 1)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 298, 1, 32)        6080      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 296, 1, 32)        3104      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 294, 1, 32)        3104      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 292, 1, 32)        3104      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 290, 1, 32)        3104      
_________________________________________________________________
dropout_2 (Dropout)          (None, 290, 1, 32)        0         
__________

## Model 3 không tăng số tầng Conv2D mà tăng lên độ rộng của nó
Kết quả đạt được đã cho thấy hiệu quả được tăng lên khi tăng độ rộng các tầng
### Kết luận: Tăng độ rộng từng tầng Conv2D có khả năng nâng cao hiệu quả của mô hình

In [8]:
model3 = Sequential()

model3.add(Reshape((shape[1], shape[2], 1), input_shape=(shape[1], shape[2])))

model3.add(Conv2D(128, (3, 63), activation='relu'))
model3.add(Conv2D(128, (3, 1), activation='relu'))
model3.add(Conv2D(128, (3, 1), activation='relu'))

model3.add(Dropout(0.25))
model3.add(GlobalMaxPooling2D())

model3.add(Dense(128))
model3.add(Dense(1, activation='sigmoid'))

model3.summary()
model3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model3.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

y_pred3 = model3.predict_classes(X_test)
print_result(y_pred3, y_test, 'CNN Conv2d: ')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 300, 63, 1)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 298, 1, 128)       24320     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 296, 1, 128)       49280     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 294, 1, 128)       49280     
_________________________________________________________________
dropout_3 (Dropout)          (None, 294, 1, 128)       0         
_________________________________________________________________
global_max_pooling2d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
__________

## Model 4 kết hợp model 2 và 3, vừa tăng số tầng Conv2D vừa tăng độ rộng của nó
Kết quả đạt được đã không được cải thiện thêm

In [9]:
model4 = Sequential()

model4.add(Reshape((shape[1], shape[2], 1), input_shape=(shape[1], shape[2])))

model4.add(Conv2D(128, (3, 63), activation='relu'))
model4.add(Conv2D(128, (3, 1), activation='relu'))
model4.add(Conv2D(128, (3, 1), activation='relu'))
model4.add(Conv2D(128, (3, 1), activation='relu'))
model4.add(Conv2D(128, (3, 1), activation='relu'))

model4.add(Dropout(0.25))
model4.add(GlobalMaxPooling2D())

model4.add(Dense(128))
model4.add(Dense(1, activation='sigmoid'))

model4.summary()
model4.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model4.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

y_pred4 = model4.predict_classes(X_test)
print_result(y_pred4, y_test, 'CNN Conv2d: ')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 300, 63, 1)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 298, 1, 128)       24320     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 296, 1, 128)       49280     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 294, 1, 128)       49280     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 292, 1, 128)       49280     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 290, 1, 128)       49280     
_________________________________________________________________
dropout_4 (Dropout)          (None, 290, 1, 128)       0         
__________